In [14]:
import os
import time
import torch
import random
import numpy as np
import open3d as o3d
from SimAttention.dataloader import ModelNetDataSet

root = '/home/akira/下载/Pointnet2_PyTorch-master/byol_pcl/data/modelnet40_normal_resampled'
data = ModelNetDataSet(root)
DataLoader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True)

demo = 0
for data, cls in DataLoader:
    demo = data
    break
numpy_demo = demo.numpy()

The size of train data is 9843


In [10]:
def visualize_samples(d_sample):
    print("Points in downsample set: ", len(d_sample))
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

In [68]:
# chair
visualize_samples(numpy_demo[3])

Points in downsample set:  10000


In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [70]:
def index_points(points, idx):
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)

def b_FPS(xyz, npoint):
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        distance = torch.min(distance, dist)
        farthest = torch.max(distance, -1)[1]
    return centroids, index_points(xyz, centroids)

def k_patch(x, k=2048, n_patch=8, n_points=1024):
    # 首先是用fps算法选择出来8个中心点，下面是中心点的索引
    patch_centers_index, _ = b_FPS(x, n_patch) # torch.Size([4, 8])
    print("patch_center_index shape: ", patch_centers_index.shape)
    # knn部分
    x = x.permute(0, 2, 1)
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
    print('pairwise_distance shape: ', pairwise_distance.shape) # torch.Size([4, 10000, 10000])
    # (batch_size, num_points, k) ---> bs, n_patch, k 
    idx = pairwise_distance.topk(k=k, dim=-1)[1][:,patch_centers_index[1],:]
    # idx = pairwise_distance.topk(k=k, dim=-1)[1]
    print('idx shape: ', idx.shape) # torch.Size([4, 8, 2048])
    idx = idx.permute(0, 2, 1)  # 4, 2048, 8
    # new patch points
    new_patch = torch.zeros([8, 4, 1024, 3]).to(device)
    for i in range(n_patch):
        patch_idx = idx[:,i,:].reshape(x.shape[0], -1)
        _, patch_points = b_FPS(index_points(x.permute(0, 2, 1), patch_idx), n_points)
        new_patch[i] = patch_points
    new_patch = new_patch.permute(1, 0, 2, 3)
#     print('new_patch shape: ', new_patch.shape)
    return x, new_patch

In [71]:
x, k_result = k_patch(demo)
k = k_result[3][0]
x = x[3].cpu().numpy()
k = k.cpu().numpy()
visualize_samples(x)

patch_center_index shape:  torch.Size([4, 8])
pairwise_distance shape:  torch.Size([4, 10000, 10000])
idx shape:  torch.Size([4, 8, 2048])
Points in downsample set:  3


RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

In [72]:
visualize_samples(k)

Points in downsample set:  1024
